<a href="https://colab.research.google.com/github/JanePer/AI---PORTFOLIO/blob/main/Task4(Lite)_%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lite**

#Классификация текстов писателей

**1. Импорт библиотек.**

In [ ]:
from google.colab import files
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
%matplotlib inline

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Embedding, SpatialDropout1D, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**2. Загрузка базы текстов.**

In [ ]:
!rm -R '/content/texts' #remove texts directory
!unzip -q '/content/drive/MyDrive/Colab Notebooks/Тексты писателей.zip' -d '/content/texts' #unzip to created `texts` directory

rm: cannot remove '/content/texts': No such file or directory


In [ ]:
def readtext(file_name): #read all text in 1 raw
  file=open(file_name, 'r')
  text=file.read()
  text=text.replace("\n"," ")
  return text

authors = ["О. Генри", "Стругацкие", "Булгаков", "Саймак", "Фрай", "Брэдберри"] #list of unique authors
authors_qty=len(authors)

In [ ]:
train_text=[]
test_text=[]

for i in authors:
  for t in os.listdir('/content/texts/Тексты писателей'):
    if i in t:
      if "Обучающая" in t:
        train_text.append(readtext('/content/texts/Тексты писателей/'+t))
      else:
        test_text.append(readtext('/content/texts/Тексты писателей/'+t))
print(len(train_text))
print(len(train_text[0]))

6
1049517


**А. Запустите нейронку c bag of words (01) при разных maxWordsCount**

***- maxWordsCount=100***

In [ ]:
max_words_qty=100
toks=Tokenizer(num_words=max_words_qty, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=" ",
                 oov_token='unknown', char_level=False) #create tokenizer model based on words frequency
toks.fit_on_texts(train_text) #apply tokens to train texts
freq_items=list(toks.word_index.items()) #extracting words with their indexes

train_word_indexes=toks.texts_to_sequences(train_text) #convert train texts into indexes
test_word_indexes=toks.texts_to_sequences(test_text) #convert test texts into indexes

In [ ]:
def getSetIndexes(wordIndexes, xLen, step): #cut separate author's text into xLen samples
  x_sample=[]
  wordslen=len(wordIndexes)
  index=0
  while (index+xLen)<=wordslen:
    x_sample.append(wordIndexes[index:index+xLen])
    index+=step
  return x_sample

def getAllSetsIndexes(wordIndexes, xLen, step): #create X set cut into defined length of samples and Y set with correspond author within all authors
  classes=len(wordIndexes) #6 authors
  class_x_sample=[]
  for w in wordIndexes: #take each list of 6 lists
    class_x_sample.append(getSetIndexes(w, xLen, step)) #cut each text list into samples and append to summary list of lists

  X_sam=[]
  Y_sam=[]
  for c in range(classes): #within 6 authors
    cx=class_x_sample[c] #all cut samples of each author's list
    for x in range(len(cx)): #within qty of all samples of each author
      X_sam.append(cx[x]) #append all cut samples to final list
      Y_sam.append(utils.to_categorical(c, classes)) #append all correspond author's vectors to final list

  X_sam=np.array(X_sam)
  Y_sam=np.array(Y_sam)
  return (X_sam, Y_sam)

In [ ]:
xLen=1000
step=100
x_train, y_train=getAllSetsIndexes(train_word_indexes, xLen, step)
x_test, y_test=getAllSetsIndexes(test_word_indexes, xLen, step)

X01=toks.sequences_to_matrix(x_train.tolist()) #convert indexes into bag of words vectors
TX01=toks.sequences_to_matrix(x_test.tolist())

In [ ]:
model100=Sequential()
model100.add(Dense(400, input_dim=max_words_qty, activation='relu'))
model100.add(Dropout(0.2))
model100.add(Dense(6, activation='sigmoid'))

model100.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model100.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 400)               40400     
_________________________________________________________________
dropout (Dropout)            (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 2406      
Total params: 42,806
Trainable params: 42,806
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history100=model100.fit(X01, y_train, epochs=10, batch_size=128, validation_data=(TX01, y_test), verbose=1)

Epoch 1/10
138/138 [==============================] - 4s 4ms/step - loss: 1.1317 - accuracy: 0.5852 - val_loss: 1.2334 - val_accuracy: 0.5298
Epoch 2/10
138/138 [==============================] - 0s 4ms/step - loss: 0.8390 - accuracy: 0.6980 - val_loss: 1.2503 - val_accuracy: 0.5392
Epoch 3/10
138/138 [==============================] - 0s 3ms/step - loss: 0.7741 - accuracy: 0.7218 - val_loss: 1.2909 - val_accuracy: 0.5292
Epoch 4/10
138/138 [==============================] - 0s 4ms/step - loss: 0.7341 - accuracy: 0.7331 - val_loss: 1.2783 - val_accuracy: 0.5380
Epoch 5/10
138/138 [==============================] - 0s 3ms/step - loss: 0.6930 - accuracy: 0.7491 - val_loss: 1.2928 - val_accuracy: 0.5396
Epoch 6/10
138/138 [==============================] - 0s 3ms/step - loss: 0.6546 - accuracy: 0.7645 - val_loss: 1.3304 - val_accuracy: 0.5242
Epoch 7/10
138/138 [==============================] - 0s 3ms/step - loss: 0.6172 - accuracy: 0.7799 - val_loss: 1.3406 - val_accuracy: 0.5317
Epoch 

In [ ]:
def createTestMultiClasses(wordIndexes, xLen, step):
  classes=len(wordIndexes) #6 authors
  x_test_classes01=[]
  x_test_classes=[]
  for w in wordIndexes: #take each list of 6 lists
    sample=(getSetIndexes(w, xLen, step)) #cut each author text list into samples
    x_test_classes.append(sample) #and append to summary list of lists
    x_test_classes01.append(toks.sequences_to_matrix(sample)) #convert cut text to bag of words vector and add to the list of lists
  x_test_classes=np.array(x_test_classes, dtype=object)
  x_test_classes01=np.array(x_test_classes01, dtype=object)
  return x_test_classes01, x_test_classes

def recognizeMultiClass (model, xTest):
  total_sum=0
  for i in range(authors_qty):
    pred=model.predict(xTest[i])
    max_pred=np.argmax(pred, axis=1)
    val=[]
    for j in range(authors_qty):
      val.append(len(max_pred[max_pred==j])/len(xTest[i]))
    total_sum+=len(max_pred[max_pred==i])
    recognizedClass=np.argmax(val)

    isrecognized="Wrong"
    if recognizedClass==i:
      isrecognized="Correct!"
    str1= "Author: "+authors[i]+ " "*(11-len(authors[i]))+str(int(100*val[i]))+"% were cerognized for "+authors[recognizedClass]
    print(str1, " "*(55-len(str1)), isrecognized, sep='')

  sum_count=0
  for i in range(authors_qty):
    sum_count+=len(xTest[i])
  print('\n')
  print("Average % of prediction: ", int(100*total_sum/sum_count),"%", sep='')

  return total_sum/sum_count

In [ ]:
x_test01,x2=createTestMultiClasses(test_word_indexes, xLen, step)

In [ ]:
score100=recognizeMultiClass(model100, x_test01)

Author: О. Генри   54% were cerognized for О. Генри    Correct!
Author: Стругацкие 49% were cerognized for Стругацкие  Correct!
Author: Булгаков   56% were cerognized for Булгаков    Correct!
Author: Саймак     35% were cerognized for Саймак      Correct!
Author: Фрай       72% were cerognized for Фрай        Correct!
Author: Брэдберри  31% were cerognized for Стругацкие  Wrong


Average % of prediction: 53%


***- maxWordsCount=1000***

In [ ]:
max_words_qty=1000
toks=Tokenizer(num_words=max_words_qty, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=" ",
                 oov_token='unknown', char_level=False) #create tokenizer model based on words frequency
toks.fit_on_texts(train_text) #apply tokens to train texts
freq_items=list(toks.word_index.items()) #extracting words with their indexes

train_word_indexes=toks.texts_to_sequences(train_text) #convert train texts into indexes
test_word_indexes=toks.texts_to_sequences(test_text)

x_train, y_train=getAllSetsIndexes(train_word_indexes, xLen, step)
x_test, y_test=getAllSetsIndexes(test_word_indexes, xLen, step)

X01=toks.sequences_to_matrix(x_train.tolist()) #convert indexes into bag of words vectors
TX01=toks.sequences_to_matrix(x_test.tolist())

In [ ]:
model1K=Sequential()
model1K.add(Dense(400, input_dim=max_words_qty, activation='relu'))
model1K.add(Dropout(0.2))
model1K.add(Dense(6, activation='sigmoid'))

model1K.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1K.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 400)               400400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 2406      
Total params: 402,806
Trainable params: 402,806
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history1K=model1K.fit(X01, y_train, epochs=10, batch_size=128, validation_data=(TX01, y_test), verbose=1)

Epoch 1/10
138/138 [==============================] - 1s 5ms/step - loss: 0.1727 - accuracy: 0.9506 - val_loss: 0.3871 - val_accuracy: 0.8661
Epoch 2/10
138/138 [==============================] - 1s 4ms/step - loss: 0.0057 - accuracy: 0.9999 - val_loss: 0.4041 - val_accuracy: 0.8554
Epoch 3/10
138/138 [==============================] - 0s 3ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.3978 - val_accuracy: 0.8639
Epoch 4/10
138/138 [==============================] - 0s 3ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.4161 - val_accuracy: 0.8584
Epoch 5/10
138/138 [==============================] - 0s 3ms/step - loss: 8.1984e-04 - accuracy: 1.0000 - val_loss: 0.4222 - val_accuracy: 0.8603
Epoch 6/10
138/138 [==============================] - 1s 4ms/step - loss: 5.4956e-04 - accuracy: 1.0000 - val_loss: 0.4340 - val_accuracy: 0.8590
Epoch 7/10
138/138 [==============================] - 1s 4ms/step - loss: 4.0574e-04 - accuracy: 1.0000 - val_loss: 0.4329 - val_accuracy: 0

In [ ]:
x_test01,x2=createTestMultiClasses(test_word_indexes, xLen, step)
score1K=recognizeMultiClass(model1K, x_test01)

Author: О. Генри   94% were cerognized for О. Генри    Correct!
Author: Стругацкие 79% were cerognized for Стругацкие  Correct!
Author: Булгаков   86% were cerognized for Булгаков    Correct!
Author: Саймак     73% were cerognized for Саймак      Correct!
Author: Фрай       91% were cerognized for Фрай        Correct!
Author: Брэдберри  88% were cerognized for Брэдберри   Correct!


Average % of prediction: 86%


***- maxWordsCount=10000***

In [ ]:
max_words_qty=10000
toks=Tokenizer(num_words=max_words_qty, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=" ",
                 oov_token='unknown', char_level=False) #create tokenizer model based on words frequency
toks.fit_on_texts(train_text) #apply tokens to train texts
freq_items=list(toks.word_index.items()) #extracting words with their indexes

train_word_indexes=toks.texts_to_sequences(train_text) #convert train texts into indexes
test_word_indexes=toks.texts_to_sequences(test_text)

x_train, y_train=getAllSetsIndexes(train_word_indexes, xLen, step)
x_test, y_test=getAllSetsIndexes(test_word_indexes, xLen, step)

X01=toks.sequences_to_matrix(x_train.tolist()) #convert indexes into bag of words vectors
TX01=toks.sequences_to_matrix(x_test.tolist())

In [ ]:
model10K=Sequential()
model10K.add(Dense(400, input_dim=max_words_qty, activation='relu'))
model10K.add(Dropout(0.2))
model10K.add(Dense(6, activation='sigmoid'))

model10K.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model10K.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 400)               4000400   
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 2406      
Total params: 4,002,806
Trainable params: 4,002,806
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history10K=model10K.fit(X01, y_train, epochs=10, batch_size=128, validation_data=(TX01, y_test), verbose=1)

Epoch 1/10
138/138 [==============================] - 2s 9ms/step - loss: 0.0615 - accuracy: 0.9810 - val_loss: 0.3227 - val_accuracy: 0.8796
Epoch 2/10
138/138 [==============================] - 1s 7ms/step - loss: 1.4117e-04 - accuracy: 1.0000 - val_loss: 0.3140 - val_accuracy: 0.8824
Epoch 3/10
138/138 [==============================] - 1s 7ms/step - loss: 6.8025e-05 - accuracy: 1.0000 - val_loss: 0.3112 - val_accuracy: 0.8844
Epoch 4/10
138/138 [==============================] - 1s 7ms/step - loss: 4.3145e-05 - accuracy: 1.0000 - val_loss: 0.3083 - val_accuracy: 0.8847
Epoch 5/10
138/138 [==============================] - 1s 7ms/step - loss: 2.8136e-05 - accuracy: 1.0000 - val_loss: 0.3094 - val_accuracy: 0.8842
Epoch 6/10
138/138 [==============================] - 1s 7ms/step - loss: 2.0106e-05 - accuracy: 1.0000 - val_loss: 0.3081 - val_accuracy: 0.8854
Epoch 7/10
138/138 [==============================] - 1s 7ms/step - loss: 1.5973e-05 - accuracy: 1.0000 - val_loss: 0.3079 - val

In [ ]:
x_test01,x2=createTestMultiClasses(test_word_indexes, xLen, step)
score10K=recognizeMultiClass(model10K, x_test01)

Author: О. Генри   95% were cerognized for О. Генри    Correct!
Author: Стругацкие 86% were cerognized for Стругацкие  Correct!
Author: Булгаков   76% were cerognized for Булгаков    Correct!
Author: Саймак     80% were cerognized for Саймак      Correct!
Author: Фрай       92% were cerognized for Фрай        Correct!
Author: Брэдберри  97% were cerognized for Брэдберри   Correct!


Average % of prediction: 88%


***- maxWordsCount=50000***

In [ ]:
#max_words_qty=50000
#xLen=1000
#step=100
#toks=Tokenizer(num_words=max_words_qty, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=" ",
                 #oov_token='unknown', char_level=False) #create tokenizer model based on words frequency
#toks.fit_on_texts(train_text) #apply tokens to train texts
#freq_items=list(toks.word_index.items()) #extracting words with their indexes

#x_train, y_train=getAllSetsIndexes(train_word_indexes, xLen, step)
#x_test, y_test=getAllSetsIndexes(test_word_indexes, xLen, step)

#X01=toks.sequences_to_matrix(x_train.tolist()) #convert indexes into bag of words vectors
#TX01=toks.sequences_to_matrix(x_test.tolist())

In [ ]:
#model50K=Sequential()
#model50K.add(Dense(400, input_dim=max_words_qty, activation='relu'))
#model50K.add(Dropout(0.2))
#model50K.add(Dense(6, activation='sigmoid'))

#model50K.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model50K.summary()

In [ ]:
#history50K=model50K.fit(X01, y_train, epochs=10, batch_size=128, validation_data=(TX01, y_test), verbose=1)

In [ ]:
#x_test01,x2=createTestMultiClasses(test_word_indexes, xLen, step)
#score50K=recognizeMultiClass(model50K, x_test01)

***Unfortinately RAM limit does not allow me to train model50K.***

**Б. Запустите нейронку c bag of words (01) при maxWordsCount = 10000 и разных архитектурах**

**- Поменяйте количество нейронов в слоях**

In [ ]:
max_words_qty=10000
xLen=1000
step=100
toks=Tokenizer(num_words=max_words_qty, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=" ",
                 oov_token='unknown', char_level=False) #create tokenizer model based on words frequency
toks.fit_on_texts(train_text) #apply tokens to train texts
freq_items=list(toks.word_index.items()) #extracting words with their indexes

train_word_indexes=toks.texts_to_sequences(train_text) #convert train texts into indexes
test_word_indexes=toks.texts_to_sequences(test_text)

x_train, y_train=getAllSetsIndexes(train_word_indexes, xLen, step)
x_test, y_test=getAllSetsIndexes(test_word_indexes, xLen, step)

X01=toks.sequences_to_matrix(x_train.tolist()) #convert indexes into bag of words vectors
TX01=toks.sequences_to_matrix(x_test.tolist())

In [ ]:
model10K_800N=Sequential()
model10K_800N.add(Dense(800, input_dim=max_words_qty, activation='relu'))
model10K_800N.add(Dropout(0.2))
model10K_800N.add(Dense(6, activation='sigmoid'))

model10K_800N.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model10K_800N.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 800)               8000800   
_________________________________________________________________
dropout_3 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 4806      
Total params: 8,005,606
Trainable params: 8,005,606
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history10K_800N=model10K_800N.fit(X01, y_train, epochs=10, batch_size=128, validation_data=(TX01, y_test), verbose=1)

Epoch 1/10
138/138 [==============================] - 2s 10ms/step - loss: 0.0617 - accuracy: 0.9806 - val_loss: 0.3498 - val_accuracy: 0.8708
Epoch 2/10
138/138 [==============================] - 1s 7ms/step - loss: 9.3949e-05 - accuracy: 1.0000 - val_loss: 0.3350 - val_accuracy: 0.8760
Epoch 3/10
138/138 [==============================] - 1s 7ms/step - loss: 4.9213e-05 - accuracy: 1.0000 - val_loss: 0.3269 - val_accuracy: 0.8786
Epoch 4/10
138/138 [==============================] - 1s 7ms/step - loss: 3.0807e-05 - accuracy: 1.0000 - val_loss: 0.3253 - val_accuracy: 0.8787
Epoch 5/10
138/138 [==============================] - 1s 7ms/step - loss: 1.9995e-05 - accuracy: 1.0000 - val_loss: 0.3143 - val_accuracy: 0.8833
Epoch 6/10
138/138 [==============================] - 1s 7ms/step - loss: 1.4273e-05 - accuracy: 1.0000 - val_loss: 0.3074 - val_accuracy: 0.8860
Epoch 7/10
138/138 [==============================] - 1s 7ms/step - loss: 1.0682e-05 - accuracy: 1.0000 - val_loss: 0.3051 - va

In [ ]:
x_test01,x2=createTestMultiClasses(test_word_indexes, xLen, step)
score10K_800N=recognizeMultiClass(model10K_800N, x_test01)

Author: О. Генри   95% were cerognized for О. Генри    Correct!
Author: Стругацкие 83% were cerognized for Стругацкие  Correct!
Author: Булгаков   80% were cerognized for Булгаков    Correct!
Author: Саймак     86% were cerognized for Саймак      Correct!
Author: Фрай       92% were cerognized for Фрай        Correct!
Author: Брэдберри  95% were cerognized for Брэдберри   Correct!


Average % of prediction: 89%


**- Поменяйте количество слоев**

In [ ]:
model10K_3L=Sequential()
model10K_3L.add(Dense(200, input_dim=max_words_qty, activation='relu'))
model10K_3L.add(Dense(100, activation='relu'))
model10K_3L.add(Dense(100, activation='relu'))
model10K_3L.add(Dropout(0.2))
model10K_3L.add(Dense(6, activation='sigmoid'))

model10K_3L.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model10K_3L.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 200)               2000200   
_________________________________________________________________
dense_9 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 606       
Total params: 2,031,006
Trainable params: 2,031,006
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history10K_3L=model10K_3L.fit(X01, y_train, epochs=10, batch_size=128, validation_data=(TX01, y_test), verbose=1)

Epoch 1/10
138/138 [==============================] - 1s 8ms/step - loss: 0.1095 - accuracy: 0.9615 - val_loss: 0.2813 - val_accuracy: 0.8984
Epoch 2/10
138/138 [==============================] - 1s 7ms/step - loss: 1.0433e-04 - accuracy: 1.0000 - val_loss: 0.2950 - val_accuracy: 0.8925
Epoch 3/10
138/138 [==============================] - 1s 7ms/step - loss: 7.3613e-05 - accuracy: 1.0000 - val_loss: 0.3060 - val_accuracy: 0.8869
Epoch 4/10
138/138 [==============================] - 1s 7ms/step - loss: 5.0164e-05 - accuracy: 1.0000 - val_loss: 0.3193 - val_accuracy: 0.8863
Epoch 5/10
138/138 [==============================] - 1s 7ms/step - loss: 2.9374e-05 - accuracy: 1.0000 - val_loss: 0.2968 - val_accuracy: 0.8934
Epoch 6/10
138/138 [==============================] - 1s 7ms/step - loss: 1.5798e-05 - accuracy: 1.0000 - val_loss: 0.3094 - val_accuracy: 0.8884
Epoch 7/10
138/138 [==============================] - 1s 7ms/step - loss: 1.1317e-05 - accuracy: 1.0000 - val_loss: 0.2949 - val

In [ ]:
x_test01,x2=createTestMultiClasses(test_word_indexes, xLen, step)
score10K_3L=recognizeMultiClass(model10K_3L, x_test01)

Author: О. Генри   94% were cerognized for О. Генри    Correct!
Author: Стругацкие 83% were cerognized for Стругацкие  Correct!
Author: Булгаков   83% were cerognized for Булгаков    Correct!
Author: Саймак     86% were cerognized for Саймак      Correct!
Author: Фрай       93% were cerognized for Фрай        Correct!
Author: Брэдберри  94% were cerognized for Брэдберри   Correct!


Average % of prediction: 89%


**- Поменяйте активационные функции слоев**

*- один Dense слой*

In [ ]:
model10K_sigm=Sequential()
model10K_sigm.add(Dense(400, input_dim=max_words_qty, activation='sigmoid'))
model10K_sigm.add(Dropout(0.2))
model10K_sigm.add(Dense(6, activation='softmax'))

model10K_sigm.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model10K_sigm.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 400)               4000400   
_________________________________________________________________
dropout_5 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 2406      
Total params: 4,002,806
Trainable params: 4,002,806
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history10K_sigm=model10K_sigm.fit(X01, y_train, epochs=10, batch_size=128, validation_data=(TX01, y_test), verbose=1)

Epoch 1/10
138/138 [==============================] - 2s 9ms/step - loss: 0.1252 - accuracy: 0.9714 - val_loss: 0.4094 - val_accuracy: 0.8872
Epoch 2/10
138/138 [==============================] - 1s 7ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.3681 - val_accuracy: 0.8920
Epoch 3/10
138/138 [==============================] - 1s 7ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.3456 - val_accuracy: 0.8974
Epoch 4/10
138/138 [==============================] - 1s 7ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.3295 - val_accuracy: 0.8975
Epoch 5/10
138/138 [==============================] - 1s 7ms/step - loss: 7.4498e-04 - accuracy: 1.0000 - val_loss: 0.3225 - val_accuracy: 0.8964
Epoch 6/10
138/138 [==============================] - 1s 7ms/step - loss: 5.3994e-04 - accuracy: 1.0000 - val_loss: 0.3111 - val_accuracy: 0.8984
Epoch 7/10
138/138 [==============================] - 1s 7ms/step - loss: 4.1566e-04 - accuracy: 1.0000 - val_loss: 0.3033 - val_accuracy: 0

In [ ]:
x_test01,x2=createTestMultiClasses(test_word_indexes, xLen, step)
score10K_sigm=recognizeMultiClass(model10K_sigm, x_test01)

Author: О. Генри   95% were cerognized for О. Генри    Correct!
Author: Стругацкие 88% were cerognized for Стругацкие  Correct!
Author: Булгаков   80% were cerognized for Булгаков    Correct!
Author: Саймак     86% were cerognized for Саймак      Correct!
Author: Фрай       92% were cerognized for Фрай        Correct!
Author: Брэдберри  97% were cerognized for Брэдберри   Correct!


Average % of prediction: 90%


*- три Dense слоя*

In [ ]:
model10K_3L_sigm=Sequential()
model10K_3L_sigm.add(Dense(200, input_dim=max_words_qty, activation='sigmoid'))
model10K_3L_sigm.add(Dense(100, activation='sigmoid'))
model10K_3L_sigm.add(Dense(100, activation='sigmoid'))
model10K_3L_sigm.add(Dropout(0.2))
model10K_3L_sigm.add(Dense(6, activation='softmax'))

model10K_3L_sigm.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model10K_3L_sigm.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 200)               2000200   
_________________________________________________________________
dense_15 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 606       
Total params: 2,031,006
Trainable params: 2,031,006
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history10K_3L_sigm=model10K_3L_sigm.fit(X01, y_train, epochs=10, batch_size=128, validation_data=(TX01, y_test), verbose=1)

Epoch 1/10
138/138 [==============================] - 1s 8ms/step - loss: 0.8180 - accuracy: 0.7579 - val_loss: 0.4584 - val_accuracy: 0.8932
Epoch 2/10
138/138 [==============================] - 1s 6ms/step - loss: 0.0447 - accuracy: 1.0000 - val_loss: 0.3093 - val_accuracy: 0.9050
Epoch 3/10
138/138 [==============================] - 1s 6ms/step - loss: 0.0154 - accuracy: 1.0000 - val_loss: 0.2805 - val_accuracy: 0.9064
Epoch 4/10
138/138 [==============================] - 1s 7ms/step - loss: 0.0085 - accuracy: 1.0000 - val_loss: 0.2754 - val_accuracy: 0.9062
Epoch 5/10
138/138 [==============================] - 1s 7ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.2726 - val_accuracy: 0.9068
Epoch 6/10
138/138 [==============================] - 1s 7ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.2719 - val_accuracy: 0.9077
Epoch 7/10
138/138 [==============================] - 1s 6ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.2763 - val_accuracy: 0.9064
Epoch 

In [ ]:
x_test01,x2=createTestMultiClasses(test_word_indexes, xLen, step)
score10K_3L_sigm=recognizeMultiClass(model10K_3L_sigm, x_test01)

Author: О. Генри   97% were cerognized for О. Генри    Correct!
Author: Стругацкие 88% were cerognized for Стругацкие  Correct!
Author: Булгаков   85% were cerognized for Булгаков    Correct!
Author: Саймак     91% were cerognized for Саймак      Correct!
Author: Фрай       88% were cerognized for Фрай        Correct!
Author: Брэдберри  97% were cerognized for Брэдберри   Correct!


Average % of prediction: 90%


**В. Запустите нейронку c Embbedding при maxWordsCount = 10000, поменяйте размер Embedding пространства**

**- Embedding space = 10**

In [ ]:
modelE10 = Sequential()
modelE10.add(Embedding(max_words_qty, 10, input_length=1000))
modelE10.add(SpatialDropout1D(0.2))
modelE10.add(Flatten())
modelE10.add(BatchNormalization())
modelE10.add(Dense(400, activation="relu"))
modelE10.add(Dropout(0.2))
modelE10.add(BatchNormalization())
modelE10.add(Dense(6, activation='sigmoid'))

modelE10.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelE10.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 10)          100000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 1000, 10)          0         
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 10000)             40000     
_________________________________________________________________
dense_18 (Dense)             (None, 400)               4000400   
_________________________________________________________________
dropout_7 (Dropout)          (None, 400)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 400)              

In [ ]:
historyE10=modelE10.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test), verbose=1)

Epoch 1/10
138/138 [==============================] - 3s 15ms/step - loss: 0.8957 - accuracy: 0.7129 - val_loss: 1.7689 - val_accuracy: 0.3603
Epoch 2/10
138/138 [==============================] - 2s 13ms/step - loss: 0.0312 - accuracy: 0.9929 - val_loss: 1.4119 - val_accuracy: 0.4158
Epoch 3/10
138/138 [==============================] - 2s 13ms/step - loss: 0.0187 - accuracy: 0.9959 - val_loss: 1.0235 - val_accuracy: 0.6352
Epoch 4/10
138/138 [==============================] - 2s 13ms/step - loss: 0.0139 - accuracy: 0.9964 - val_loss: 0.9100 - val_accuracy: 0.6774
Epoch 5/10
138/138 [==============================] - 2s 13ms/step - loss: 0.0080 - accuracy: 0.9981 - val_loss: 0.8106 - val_accuracy: 0.7155
Epoch 6/10
138/138 [==============================] - 2s 14ms/step - loss: 0.0076 - accuracy: 0.9981 - val_loss: 0.9036 - val_accuracy: 0.7080
Epoch 7/10
138/138 [==============================] - 2s 13ms/step - loss: 0.0078 - accuracy: 0.9980 - val_loss: 0.9130 - val_accuracy: 0.7245

In [ ]:
x1, x_test_classes = createTestMultiClasses(test_word_indexes, xLen, step)
scoreE10 = recognizeMultiClass(modelE10, x_test_classes)

Author: О. Генри   82% were cerognized for О. Генри    Correct!
Author: Стругацкие 58% were cerognized for Стругацкие  Correct!
Author: Булгаков   58% were cerognized for Булгаков    Correct!
Author: Саймак     65% were cerognized for Саймак      Correct!
Author: Фрай       87% were cerognized for Фрай        Correct!
Author: Брэдберри  75% were cerognized for Брэдберри   Correct!


Average % of prediction: 72%


**- Embedding space = 50**

In [ ]:
modelE50 = Sequential()
modelE50.add(Embedding(max_words_qty, 50, input_length=1000))
modelE50.add(SpatialDropout1D(0.2))
modelE50.add(Flatten())
modelE50.add(BatchNormalization())
modelE50.add(Dense(400, activation="relu"))
modelE50.add(Dropout(0.2))
modelE50.add(BatchNormalization())
modelE50.add(Dense(6, activation='sigmoid'))

modelE50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelE50.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 50)          500000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1000, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50000)             0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 50000)             200000    
_________________________________________________________________
dense_20 (Dense)             (None, 400)               20000400  
_________________________________________________________________
dropout_8 (Dropout)          (None, 400)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 400)              

In [ ]:
historyE50=modelE50.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test), verbose=1)

Epoch 1/10
138/138 [==============================] - 6s 37ms/step - loss: 0.8505 - accuracy: 0.7776 - val_loss: 1.9939 - val_accuracy: 0.3778
Epoch 2/10
138/138 [==============================] - 5s 35ms/step - loss: 0.0097 - accuracy: 0.9995 - val_loss: 1.6373 - val_accuracy: 0.4020
Epoch 3/10
138/138 [==============================] - 5s 35ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 1.3717 - val_accuracy: 0.4925
Epoch 4/10
138/138 [==============================] - 5s 35ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 1.0552 - val_accuracy: 0.6200
Epoch 5/10
138/138 [==============================] - 5s 36ms/step - loss: 0.0014 - accuracy: 0.9999 - val_loss: 0.9303 - val_accuracy: 0.6840
Epoch 6/10
138/138 [==============================] - 5s 35ms/step - loss: 9.8977e-04 - accuracy: 1.0000 - val_loss: 0.8942 - val_accuracy: 0.7088
Epoch 7/10
138/138 [==============================] - 5s 35ms/step - loss: 7.8291e-04 - accuracy: 1.0000 - val_loss: 0.9237 - val_accuracy

In [ ]:
scoreE50 = recognizeMultiClass(modelE50, x_test_classes)

Author: О. Генри   83% were cerognized for О. Генри    Correct!
Author: Стругацкие 69% were cerognized for Стругацкие  Correct!
Author: Булгаков   55% were cerognized for Булгаков    Correct!
Author: Саймак     75% were cerognized for Саймак      Correct!
Author: Фрай       81% were cerognized for Фрай        Correct!
Author: Брэдберри  66% were cerognized for Брэдберри   Correct!


Average % of prediction: 71%


**- Embedding space = 200**

In [ ]:
modelE200 = Sequential()
modelE200.add(Embedding(max_words_qty, 200, input_length=1000))
modelE200.add(SpatialDropout1D(0.2))
modelE200.add(Flatten())
modelE200.add(BatchNormalization())
modelE200.add(Dense(400, activation="relu"))
modelE200.add(Dropout(0.2))
modelE200.add(BatchNormalization())
modelE200.add(Dense(6, activation='sigmoid'))

modelE200.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelE200.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 200)         2000000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 1000, 200)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 200000)            0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 200000)            800000    
_________________________________________________________________
dense_22 (Dense)             (None, 400)               80000400  
_________________________________________________________________
dropout_9 (Dropout)          (None, 400)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 400)              

In [ ]:
historyE200=modelE200.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test), verbose=1)

Epoch 1/10
138/138 [==============================] - 18s 123ms/step - loss: 0.7338 - accuracy: 0.8259 - val_loss: 2.0035 - val_accuracy: 0.3709
Epoch 2/10
138/138 [==============================] - 17s 124ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 1.5404 - val_accuracy: 0.3857
Epoch 3/10
138/138 [==============================] - 17s 126ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 1.2772 - val_accuracy: 0.5069
Epoch 4/10
138/138 [==============================] - 17s 126ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.9589 - val_accuracy: 0.6545
Epoch 5/10
138/138 [==============================] - 17s 124ms/step - loss: 7.6294e-04 - accuracy: 1.0000 - val_loss: 0.8639 - val_accuracy: 0.7063
Epoch 6/10
138/138 [==============================] - 17s 125ms/step - loss: 5.4173e-04 - accuracy: 1.0000 - val_loss: 0.8372 - val_accuracy: 0.7269
Epoch 7/10
138/138 [==============================] - 17s 125ms/step - loss: 4.0147e-04 - accuracy: 1.0000 - val_loss: 0.8

In [ ]:
scoreE200 = recognizeMultiClass(modelE200, x_test_classes)

Author: О. Генри   86% were cerognized for О. Генри    Correct!
Author: Стругацкие 66% were cerognized for Стругацкие  Correct!
Author: Булгаков   64% were cerognized for Булгаков    Correct!
Author: Саймак     63% were cerognized for Саймак      Correct!
Author: Фрай       84% were cerognized for Фрай        Correct!
Author: Брэдберри  71% were cerognized for Брэдберри   Correct!


Average % of prediction: 73%


In [ ]:
print("Average % of prediction for model100: ",round(score100*100,1))
print("Average % of prediction for model1K: ",round(score1K*100,1))
print("Average % of prediction for model10K: ",round(score10K*100,1))
print("Average % of prediction for model50K: unknown")
print("Average % of prediction for model10K_800N: ",round(score10K_800N*100,1))
print("Average % of prediction for model10K_3L: ",round(score10K_3L*100,1))
print("Average % of prediction for model10K_sigm: ",round(score10K_sigm*100,1))
print("Average % of prediction for model10K_3L_sigm: ",round(score10K_3L_sigm*100,1))
print("Average % of prediction for modelE10: ",round(scoreE10*100,1))
print("Average % of prediction for modelE50: ",round(scoreE50*100,1))
print("Average % of prediction for modelE200: ",round(scoreE200*100,1))

Average % of prediction for model100:  53.4
Average % of prediction for model1K:  86.7
Average % of prediction for model10K:  88.6
Average % of prediction for model50K: unknown
Average % of prediction for model10K_800N:  89.2
Average % of prediction for model10K_3L:  89.8
Average % of prediction for model10K_sigm:  90.2
Average % of prediction for model10K_3L_sigm:  90.6
Average % of prediction for modelE10:  72.7
Average % of prediction for modelE50:  71.4
Average % of prediction for modelE200:  73.6


- The more words our dictionary has the higher model's accuracy is, but possible RAM limit may disable large max_words_count. Large set of dictionary allow model to distinguish unique content.
- With this dataset just increasing number of neurons or number of layers does not make huge difference. But changing activation functions + increasing number of layers improved model for a couple of percents.
- Bag of words approach is more efficiency than embedding for multi categorizing texts.